# Workflow to Validate NISAR L2 Secular Displacement Requirement

**Original code authored by:** David Bekaert, Heresh Fattahi, Eric Fielding, and Zhang Yunjun 

Extensive modifications by Adrian Borsa and Amy Whetter 2022 <br>
Subsequent updates and reformatting by Rob Zinke, Katia Tymofyeveva and Adrian Borsa 2024 <br>
Optional corrections and plots by Emre Havazli, April 2025<br>

<div class="alert alert-warning">
Step 1 of the validation workflow is executed in the <b>ARIA_prep</b> notebook. All processing steps in this notebook MUST be run sequentially, although several are optional and can be skipped.
</div>

<hr/>

## Table of Contents: <a id='TOC'></a>

[**Environment Setup**](#setup)
- [Load Python Packages](#load_packages)
- [Define CalVal Site and Parameters](#set_calval_params)
- [Set Directories and Files](#set_directories)

[**1. Download and Prepare Interferograms**](#prep_ifg)
[Executed in ARIA_prep]

[**2. Generate Time Series from Interferograms**](#gen_ts)
- [2.1. Validate/Modify Interferogram Network](#validate_network)
- [2.2. Generate Quality Control Mask](#generate_mask)
- [2.3. Reference Interferograms To Common Lat/Lon](#common_latlon)
- [2.4. Invert for SBAS Line-of-Sight Timeseries](#invert_SBAS)

[**3. Optional Corrections**](#opt_correction)
- [3.1. Solid Earth Tide Correction](#solid_earth)
- [3.2. Tropospheric Delay Correction](#tropo_corr)
- [3.3. Phase Deramping ](#phase_deramp)
- [3.4. Topographic Residual Correction ](#topo_corr) 

[**4. Decompose InSAR and GNSS Time Series**](#decomp_ts)
- [4.1. Estimate InSAR LOS Velocities](#insar_vel)
- [4.2. Not Used](#empty)
- [4.3. Find Collocated GNSS Stations](#find_gps)  
- [4.4. Get GNSS Position Time Series](#gps_ts) 
- [4.5. Estimate GNSS LOS Velocities](#gps_vel)
- [4.6. Re-reference GNSS and InSAR](#reference)

[**5. Validation Method 1: GNSS-InSAR Direct Comparison**](#validation1)
- [5.1. Make GNSS-InSAR Velocity Residuals at GNSS Station Locations](#make_resid)
- [5.2. Make Double-differenced Velocity Residuals](#make_ddiff)
- [5.3. Secular Requirement Validation: Method 1](#secular_validation_method1)

[**6. Validation Method 2: InSAR-only Structure Function**](#validation2)
- [6.1. Read InSAR Array and Mask Pixels with No Data](#array_mask)
- [6.2. Randomly Sample and Pair Pixels](#array_sample)
- [6.3. Secular Requirement Validation: Method 2](#secular_validation_method2)

[**Appendix: Supplementary Comparisons and Plots**](#appendix)
- [A.1. Compare Raw Velocities](#plot_vel)
- [A.2. Plot Velocity Residuals](#plot_residual)
- [A.3. Plot Double-differenced Residuals](#plot_ddiff)
- [A.4. GNSS Timeseries Plots](#plot_gps_tseries)

<br>
<hr>

<a id='#setup'></a>
## Environment Setup

### Load Python Packages <a id='#load_packages'></a>

In [ ]:
import json
import numpy as np
import os
import subprocess
import warnings

from datetime import datetime as dt
from matplotlib import pyplot as plt
from mintpy.cli import view, plot_network
from mintpy.objects import gnss, timeseries
from mintpy.smallbaselineApp import TimeSeriesAnalysis
from mintpy.utils import ptime, readfile, utils as ut
from scipy import signal

from solid_utils.gnss_utils import scale_gnss_m_to_mm
from solid_utils.fitting import IterativeOutlierFit
from solid_utils.plotting import display_validation, display_validation_table
from solid_utils.sampling import SiteVelocity, load_geo, samp_pair, profile_samples, haversine_distance

#Set Global Plot Parameters
plt.rcParams.update({'font.size': 12})

### Define Calval Site and Parameters <a id='set_calval_params'></a>

In [ ]:
# Specify a calval location ID from my_sites.txt
site = 'test' 

# Choose the requirement to validate
# Options: 'Secular' 'Coseismic' 'Transient'
requirement = 'Secular' 

# What dataset are you processing?
# Options: 
#'ARIA_S1' (old directory structure for Sentinel-1 testing with aria-tools)
#'ARIA_S1_new' (new directory structure for Sentinel-1 testing with aria-tools)
dataset = 'ARIA_S1_new'
aria_gunw_version = '3_0_1'

# The date and version of this Cal/Val run
rundate = '20250430'
version = '1'

# Provide the file where you keep your customized list of sites.
custom_sites = '/home/jovyan/solid_earth_atbd_dev/ATBD_main/my_sites.txt'

# Enter a username for storing your outputs
if os.path.exists('/home/jovyan/solid_earth_atbd_dev/ATBD_main/me.txt'):
    with open('/home/jovyan/solid_earth_atbd_dev/ATBD_main/me.txt') as m:
        you = m.readline().strip()
else:
    you = input('Please type a username for your calval outputs:')
    with open ('/home/jovyan/me.txt', 'w') as m: 
        m.write(you)

# Load metadata for calval locations
with open('/home/jovyan/solid_earth_atbd_dev/ATBD_main/my_sites.txt','r') as fid:
    sitedata = json.load(fid)

# Plot parameters
vmin, vmax = -100, 100  # mm/yr
cmap = plt.get_cmap('RdBu_r')

### Set Directories and Files <a id='set_directories'></a>

In [ ]:
# Directory location for Cal/Val data (do not change)
start_directory = '/scratch/nisar-st-calval-solidearth' 

# Site directory
site_dir = os.path.join(start_directory, dataset, site)

# Working directory for calval processing
work_dir = os.path.join(site_dir, requirement, you, rundate, 'v' + version)
print("  Work directory:", work_dir)

# Directory for storing GUNW interferograms
gunw_dir = os.path.join(site_dir,'products')
print("  GUNW directory:", gunw_dir) 

# Directory for storing MintPy outputs
mintpy_dir = os.path.join(work_dir,'MintPy')
print("MintPy directory:", mintpy_dir)

# Flag missing MintPy directory
if not os.path.exists(mintpy_dir):
    print()
    print('ERROR: Stop! Your MintPy processing directory does not exist for this requirement, site, version, or date of your ATBD run.')
    print('You may need to run the prep notebook first!')
    print()
else:
    os.chdir(mintpy_dir)

# Set MintPy filenames
geom_file = os.path.join(mintpy_dir, 'inputs', 'geometryGeo.h5')
msk_file = os.path.join(mintpy_dir, 'maskTempCoh.h5')  # maskConnComp.h5
ts_file = os.path.join(mintpy_dir, 'timeseries.h5')
vel_file = os.path.join(mintpy_dir, 'velocity.h5')

<br>
<hr>

<a id='#prep_ifg'></a>
## 1. Download and Prepare Interferograms 
Executed in *ARIA_prep* notebook

<br>
<hr>

<a id='#gen_ts'></a>
## 2. Generation of Time Series from Interferograms

### 2.1. Validate/Modify Interferogram Network <a id='validate_network'></a>

Add additional parameters to config_file in order to remove selected interferograms, change minimum coherence, etc.

In [ ]:
config_file = os.path.join(mintpy_dir, sitedata['sites'][site]['calval_location'] + '.cfg')
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep modify_network'
process = subprocess.run(command, shell=True)
plot_network.main(['inputs/ifgramStack.h5'])

### 2.2. Generate Quality Control Mask <a id='generate_mask'></a>

Mask files can be can be used to mask pixels in the time-series processing. Below we generate an initial mask file `maskConnComp.h5` based on the connected components for all the interferograms, which is a metric for unwrapping quality. After time-series analysis is complete, we will calculate a mask from the temporal coherence or variation of phase or displacement with time to make `maskTempCoh.h5`.

In [ ]:
command='generate_mask.py inputs/ifgramStack.h5  --nonzero  -o maskConnComp.h5  --update'
process = subprocess.run(command, shell=True)
view.main(['maskConnComp.h5', 'mask'])

### 2.3. Reference Interferograms To Common Lat/Lon <a id='common_latlon'></a>

In [ ]:
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep reference_point'
process = subprocess.run(command, shell=True)
os.system('info.py inputs/ifgramStack.h5 | egrep "REF_"');

### 2.3.1 Change Reference Point (Optional) <a id='common_latlon_change'></a>

Use the cells below to update the reference point. Uncomment and enter the desired *new_lat* and *new_lon*, then verify the update in the output.

In [ ]:
# def update_reference_point(config_file, new_lat, new_lon):
#     # Read the file
#     with open(config_file, "r") as file:
#         lines = file.readlines()
    
#     # Update the specific key
#     for i, line in enumerate(lines):
#         if "mintpy.reference.lalo" in line:
#             lines[i] = f"mintpy.reference.lalo = {new_lat}, {new_lon}\n"
#             break  # Stop after updating the line
    
#     # Write back the modified file
#     with open(config_file, "w") as file:
#         file.writelines(lines)
    
#     print(f"Updated mintpy.reference.lalo to {new_lat}, {new_lon}")

In [ ]:
# new_lat =
# new_lon = 
# update_reference_point(config_file, new_lat, new_lon)  # New latitude and longitude

In [ ]:
# command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep reference_point'
# process = subprocess.run(command, shell=True)
# os.system('info.py inputs/ifgramStack.h5 | egrep "REF_"');

### 2.4. Invert for SBAS Line-of-Sight Timeseries <a id='invert_SBAS'></a>

In [ ]:
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep invert_network'
process = subprocess.run(command, shell=True)

<br>
<hr>

<a id='opt_correction'></a>
## 3. Optional Corrections

Phase distortions related to solid earth and ocean tidal effects, as well as those due to temporal variations in the vertical stratification of the atmosphere, can be mitigated using the approaches described below. At this point, it is expected that these corrections will not be needed to validate the mission requirements, but they may be used to produce the highest quality data products. Typically, these are applied to the estimated time series product rather than to the individual interferograms, since they are a function of the time of each radar acquisition.

### 3.1. Calculate Solid Earth Tides Correction <a id='solid_earth'></a>

Optional solid Earth tide correction is included in the ARIA GUNW products and can be extracted using the <i>ARIA_prep</i> notebook.  
Once extracted, you can calculate the correction using the cells below.

In [ ]:
ts_file = f"{mintpy_dir}/timeseries.h5"                      # Input timeseries file to apply tropospheric correction
cor_file = f"{mintpy_dir}/inputs/solidEarthTide_ARIA.h5"     # Input SET correction file

In [ ]:
# Check if input correction file exist
if os.path.isfile(cor_file):
    print(f"Found {cor_file}")
else:
    raise FileNotFoundError(f"Input {cor_file} not found")

# Check if the input time series file exist and read attributes
if os.path.isfile(ts_file):
    print(f"Found {ts_file}")
    atr = readfile.read_attribute(ts_file)
else:
    raise FileNotFoundError(f"Input {ts_file} not found")

In [ ]:
# Reference correction to the timeseries reference pixel
command = f"reference_point.py {cor_file} --lat {float(atr['REF_LAT'])} --lon {float(atr['REF_LON'])}"
process = subprocess.run(command, shell=True)

In [ ]:
# Reference correction to the timeseries reference date
command = f"reference_date.py {cor_file} --ref-date {atr['REF_DATE']}"
process = subprocess.run(command, shell=True)

In [ ]:
# Plot the corrections
view.main([cor_file, '-m', msk_file])

### 3.2 Calculate Ionosphere Correction

In [ ]:
ts_file = f"{mintpy_dir}/timeseries.h5"                        # Input timeseries file to apply tropospheric correction
cor_stack_file = f"{mintpy_dir}/inputs/ionStack.h5"            # Input ionosphere stack file
cor_file = f"{mintpy_dir}/ion.h5"                              # Ionosphere correction file (to be generated)

In [ ]:
# Check if input correction file exist
if os.path.isfile(cor_stack_file):
    print(f"Found {cor_stack_file}")
else:
    raise FileNotFoundError(f"Input {cor_stack_file} not found")

# Check if the input time series file exist and read attributes
if os.path.isfile(ts_file):
    print(f"Found {ts_file}")
    atr = readfile.read_attribute(ts_file)
else:
    raise FileNotFoundError(f"Input {ts_file} not found")

In [ ]:
# Modify ionosphere network
command = f'modify_network.py {cor_stack_file} -t {config_file}'
process = subprocess.run(command, shell=True)

In [ ]:
# Reference correction to the timeseries reference pixel
command = f"reference_point.py {cor_stack_file} --lat {float(atr['REF_LAT'])} --lon {float(atr['REF_LON'])}"
process = subprocess.run(command, shell=True)

In [ ]:
# Estimate ionospheric delay time-series via ifgram_inversion.py
command = f'ifgram_inversion.py {cor_stack_file} --dset unwrapPhase --weight-func no --update'
process = subprocess.run(command, shell=True)

In [ ]:
# Reference correction to the timeseries reference date
command = f"reference_date.py {cor_file} --ref-date {atr['REF_DATE']}"
process = subprocess.run(command, shell=True)

In [ ]:
# Plot the corrections
view.main([cor_file, '-m', msk_file])

### 3.3. Calculate Tropospheric Delay Correction <a id='tropo_corr'></a>

Optional atmospheric correction utilizes the PyAPS (Jolivet et al., 2011, Jolivet and Agram, 2012) module within GIAnT (or eventually a merged replacement for GIAnT and MintPy). PyAPS is well documented, maintained and can be freely downloaded. PyAPS is included in GIAnT distribution). PyAPS currently includes support for ECMWF’s ERA-Interim, NOAA’s NARR and NASA’s MERRA weather models. A final selection of atmospheric models to be used for operational NISAR processing will be done during Phase C.

[T]ropospheric delay maps are produced from atmospheric data provided by Global Atmospheric Models. This method aims to correct differential atmospheric delay correlated with the topography in interferometric phase measurements. Global Atmospheric Models (hereafter GAMs)... provide estimates of the air temperature, the atmospheric pressure and the humidity as a function of elevation on a coarse resolution latitude/longitude grid. In PyAPS, we use this 3D distribution of atmospheric variables to determine the atmospheric phase delay on each pixel of each interferogram.

The absolute atmospheric delay is computed at each SAR acquisition date. For a pixel a_i at an elevation z at acquisition date i, the four surrounding grid points are selected and the delays for their respective elevations are computed. The resulting delay at the pixel a_i is then the bilinear interpolation between the delays at the four grid points. Finally, we combine the absolute delay maps of the InSAR partner images to produce the differential delay maps used to correct the interferograms.

REFERENCE : https://github.com/insarlab/pyaps#2-account-setup-for-era5
<br> Read Section 2 for ERA5 [link above] to create an account on the CDS website.

In [ ]:
# if 'do_tropo_correction' in sitedata['sites']:
#     do_tropo_correction = sitedata['sites'][site]['do_tropo_correction']
# else:
#     do_tropo_correction = 'False'

# if do_tropo_correction == 'True':
#     if not Use_Staged_Data and not os.path.exists(os.path.expanduser("~")+'/.cdsapirc'):
#         print('NEEDED to download ERA5, link: https://cds.climate.copernicus.eu/user/register')
#         UID = input('Please type your CDS_UID:')
#         CDS_API = input('Please type your CDS_API:')
        
#         cds_tmp = '''url: https://cds.climate.copernicus.eu/api
#         key: {UID}:{CDS_API}'''.format(UID=UID, CDS_API=CDS_API)
#         os.system('echo "{cds_tmp}" > ~/.cdsapirc; chmod 600 ~/.cdsapirc'.format(cds_tmp = str(cds_tmp)))
    
#     command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep correct_troposphere'
#     process = subprocess.run(command, shell=True)
    
#     view.main(['inputs/ERA5.h5'])
#     timeseries_filename = 'timeseries_ERA5.h5'
# else:
#     timeseries_filename = 'timeseries.h5'

<div class="alert alert-block alert-info">
    <b>Note for HRRR:</b> The HRRR dataset is included in ARIA GUNW products and can be extracted using the <i>ARIA_prep</i> notebook with the <code>solid_earth_atbd_dev</code> environment. <br><br>
    If you already have the <code>HRRR_ARIA.h5</code> file, you can apply HRRR corrections using the cells below.
</div>

In [ ]:
ts_file = f"{mintpy_dir}/timeseries.h5"                      # Input timeseries file to apply tropospheric correction
cor_file = f"{mintpy_dir}/inputs/HRRR_ARIA.h5"               # Input SET correction file

In [ ]:
# Check if input correction file exist
if os.path.isfile(cor_file):
    print(f"Found {cor_file}")
else:
    raise FileNotFoundError(f"Input {cor_file} not found")

# Check if the input time series file exist and read attributes
if os.path.isfile(ts_file):
    print(f"Found {ts_file}")
    atr = readfile.read_attribute(ts_file)
else:
    raise FileNotFoundError(f"Input {ts_file} not found")

In [ ]:
# Reference correction to the timeseries reference pixel
command = f"reference_point.py {cor_file} --lat {float(atr['REF_LAT'])} --lon {float(atr['REF_LON'])}"
process = subprocess.run(command, shell=True)

In [ ]:
# Reference correction to the timeseries reference date
command = f"reference_date.py {cor_file} --ref-date {atr['REF_DATE']}"
process = subprocess.run(command, shell=True)

In [ ]:
# Plot the corrections
view.main([cor_file, '-m', msk_file])

## Apply Optional Corrections

#### Solid Earth Tides

In [ ]:
# Input files:
input_timeseries = f"{mintpy_dir}/timeseries.h5"
input_cor = f"{mintpy_dir}/inputs/solidEarthTide_ARIA.h5"

# Output files:
timeseries_filename = f"{mintpy_dir}/timeseries_SET.h5"
velocity_filename = f"{mintpy_dir}/velocity_SET.h5"

In [ ]:
# Apply SET Correction
command = f'diff.py {input_timeseries} {input_cor} -o {timeseries_filename} --force'
process = subprocess.run(command, shell=True)

In [ ]:
# Estimate velocity
command = f'timeseries2velocity.py {timeseries_filename} -o {velocity_filename}'
process = subprocess.run(command, shell=True)

In [ ]:
view.main([velocity_filename, 'velocity', '-m', msk_file, '-u', 'mm', '-v', str(vmin), str(vmax), '--title', f"{velocity_filename.split('/')[-1]}"])

#### Ionosphere

In [ ]:
# Input files:
input_timeseries = f"{mintpy_dir}/timeseries_SET.h5"
input_cor = f"{mintpy_dir}/ion.h5"

# Output file:
timeseries_filename = f"{mintpy_dir}/timeseries_SET_iono.h5"
velocity_filename = f"{mintpy_dir}/velocity_SET_iono.h5"

In [ ]:
# Apply ionopshere correction
command = f'diff.py {input_timeseries} {input_cor} -o {timeseries_filename} --force'
process = subprocess.run(command, shell=True)

In [ ]:
# Estimate velocity
command = f'timeseries2velocity.py {timeseries_filename} -o {velocity_filename}'
process = subprocess.run(command, shell=True)

In [ ]:
view.main([velocity_filename, 'velocity', '-m', msk_file, '-u', 'mm', '-v', str(vmin), str(vmax), '--title', f"{velocity_filename.split('/')[-1]}"])

#### Troposphere

In [ ]:
# Input files:
input_timeseries = f"{mintpy_dir}/timeseries_SET_iono.h5"
input_cor = f"{mintpy_dir}/inputs/HRRR_ARIA.h5"

# Output file:
timeseries_filename = f"{mintpy_dir}/timeseries_SET_iono_HRRR.h5"
velocity_filename = f"{mintpy_dir}/velocity_SET_iono_HRRR.h5"

In [ ]:
# Apply troposphere correction
command = f'diff.py {input_timeseries} {input_cor} -o {timeseries_filename} --force'
process = subprocess.run(command, shell=True)

In [ ]:
# Estimate velocity
command = f'timeseries2velocity.py {timeseries_filename} -o {velocity_filename}'
process = subprocess.run(command, shell=True)

In [ ]:
view.main([velocity_filename, 'velocity', '-m', msk_file, '-u', 'mm', '-v', str(vmin), str(vmax), '--title', f"{velocity_filename.split('/')[-1]}"])

### 3.4. Phase Deramping <a id='phase_deramp'></a>

In [ ]:
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep deramp'
process = subprocess.run(command, shell=True)

### 3.5. Topographic Residual Correction <a id='topo_corr'></a>

In [ ]:
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep correct_topography'
process = subprocess.run(command, shell=True)

<div class="alert alert-block alert-info">
    <b>Note:</b> The rest of the notebook is using the variable <code>timeseries_filename</code> as InSAR result. <b>Make sure <code>timeseries_filename</code> is set to the correct file</b>
</div>

In [ ]:
print(timeseries_filename)

<br>
<hr>

<a id='decomp_ts'></a>
## 4. Estimate InSAR and GNSS Velocities
The approach that will be used for the generation of NISAR L3 products for Requirements 660 and 663 allows for an explicit inclusion of key basis functions (e.g., Heaviside functions, secular rate, etc.) in the InSAR inversion. Modifications to this algorithm may be identified and implemented in response to NISAR Phase C activities. 

### 4.1. Estimate InSAR LOS Velocities <a id='insar_vel'></a>

Given a time series of InSAR LOS displacements, the observations for a given pixel, $U(t)$, can be parameterized as:

$$U(t) = a \;+\; vt \;+\; c_1 cos (\omega_1t - \phi_{1,}) \;+\; c_2 cos (\omega_2t - \phi_2) \;+\; \sum_{j=1}^{N_{eq}} \left( h_j+f_j F_j (t-t_j) \right)H(t - t_j) \;+\; \frac{B_\perp (t)}{R sin \theta}\delta z \;+\; residual$$ 

which includes a constant offset $(a)$, velocity $(v)$, and amplitudes $(c_j)$ and phases $(\phi_j)$ of annual $(\omega_1)$ and semiannual $(\omega_2)$ sinusoidal terms.  Where needed we can include additional complexity, such as coseismic and postseismic processes parameterized by Heaviside (step) functions $H$ and postseismic functions $F$ (the latter typically exponential and/or logarithmic).   $B_\perp(t)$, $R$, $\theta$, and $\delta z$ are, respectively, the perpendicular component of the interferometric baseline relative to the first date, slant range distance, incidence angle and topography error correction for the given pixel. 

Thus, given either an ensemble of interferograms or the output of SBAS (displacement vs. time), we can write the LSQ problem as 

$$ \textbf{G}\textbf{m} = \textbf{d}$$

where $\textbf{G}$ is the design matrix (constructed out of the different functional terms in Equation 2 evaluated either at the SAR image dates for SBAS output, or between the dates spanned by each pair for interferograms), $\textbf{m}$ is the vector of model parameters (the coefficients in Equation 2) and $\textbf{d}$ is the vector of observations.  (In the case of GNSS displacement timeseries, $\textbf{G}, \textbf{d}, \textbf{m}$ are constructed using values evaluated at single epochs corresponding to the GNSS solution times, otherwise the estimate is the same.) 

With this formulation, we can obtain InSAR and GNSS velocity estimates and their formal uncertainties (including in areas where zero deformation is expected). The default InSAR (and GNSS) velocity fit in MintPy is to estimate a mean linear velocity $(v)$ in in the equation. To estimate a secular deformation rate, one can also estimate additional parameters for annual and semi-annual terms to account for seasonal periodic fluctuations in position. Below, we define a set of basis functions that MintPy will invert for, for both InSAR and GNSS data.

<div class="alert alert-warning">
If the time-series span less than one year, then annual and semi-annual fuctions might not be reliably fit for noisy data. In that case, the periodic functions will not be included in these analyses.
</div>

In [ ]:
# Specify basis functions
# Changing these might change the meaning of this exercise! Use caution if altering.
ts_functions = {
                'polynomial': 1,
                'periodic': [1.0, 0.5],  # Periodic terms are in units of years
                'stepDate': [],
                'polyline': [],
                'exp': {},
                'log': {}
}

# Determine time span to check whether fitting of periodic functions is appropriate
ts_metadata = readfile.read_attribute(timeseries_filename)
start_date = dt.strptime(sitedata['sites'][site]['download_start_date'], '%Y%m%d')
end_date = dt.strptime(sitedata['sites'][site]['download_end_date'], '%Y%m%d')
ts_timespan = (end_date - start_date).days

if ts_timespan < 365*2:
    print('Time span is {:d} days'.format(ts_timespan))
    if ts_timespan < 365:
        ts_functions['periodic'] = []
        print('No periodic functions will be fit')
    else:
        warnings.warn('Warning: Periodic fits might be unreliable for series less than 2 years')


In [ ]:
# Parse the TS model into argument string for command line execution
function_str = ''

# Loop through basis functions
for fcn in ts_functions.keys():
    arg = ts_functions[fcn]
    
    if fcn == 'polynomial':
        function_str += f' --polynomial {arg:d}'
    
    else:
        # Check whether values are passed
        if len(arg) > 0:
            # Append basis function name
            function_str += f' --{fcn:s}'

            # Append function arguments (e.g, period, decay, etc.)
            if type(arg) == list:
                function_str += ' {:s}'.format(' '.join([str(a) for a in arg]))
            elif type(arg) == dict:
                for event_date in arg.keys():
                    function_str += f' {event_date:s}'
                    for val in arg[event_date]:
                        function_str += f' {val:f}'

# Run command
command = 'timeseries2velocity.py ' + timeseries_filename + function_str
process = subprocess.run(command, shell=True)

# Load velocity file
insar_velocities,_ = readfile.read(vel_file, datasetName = 'velocity')  # read velocity file
insar_velocities = insar_velocities * 1000.  # convert InSAR velocities from m/yr to mm/yr

# Set masked pixels to NaN
msk,_ = readfile.read(msk_file)
insar_velocities[msk == 0] = np.nan
insar_velocities[insar_velocities == 0] = np.nan

Now we plot the mean linear velocity fit. The MintPy `view` module automatically reads the temporal coherence mask `maskTempCoh.h5` and applies that to mask out pixels with unreliable velocities (white).

In [ ]:
scp_args = f"velocity.h5 velocity -v {vmin} {vmax} --colormap RdBu_r --figtitle LOS_Velocity --unit mm/yr -m {msk_file}"
view.main(scp_args.split());

### 4.2. Not Used <a id='empty'></a>

### 4.3. Find Collocated GNSS Stations <a id='find_gps'></a>

The project will have access to L2 position data for continuous GNSS stations in third-party networks (e.g., NSF’s Plate Boundary Observatory, the HVO network for Hawaii, GEONET-Japan, and GEONET-New Zealand), located in target regions for NISAR solid earth calval. Station data will be post-processed by one or more analysis centers, will be freely available, and will have latencies of several days to weeks, as is the case with positions currently produced by the NSF’s GAGE Facility and separately by the University of Nevada Reno. Networks will contain one or more areas of high-density station coverage (2~20 km nominal station spacing over 100 x 100 km or more) to support validation of L2 NISAR requirements at a wide range of length scales.

MintPy supports several options for downloading and handling GNSS data. Currently supported data sets are:
* UNR National Geodetic Laboratory
* SIO/JPL MEaSUREs ESESES
* JPL SIDESHOW

In [ ]:
# GNSS processing source
if 'gnss_source' in sitedata['sites'][site]:
    gnss_source = sitedata['sites'][site]['gnss_source']
else:
    gnss_source = 'UNR'
print(f"GNSS processing source: {gnss_source:s}")

# Get analysis metadata from InSAR velocity file
insar_metadata = readfile.read_attribute(vel_file)
lat_step = float(insar_metadata['Y_STEP'])
lon_step = float(insar_metadata['X_STEP'])
(S,N,W,E) = ut.four_corners(insar_metadata)
start_date = insar_metadata.get('START_DATE', None).split('T')[0]
end_date = insar_metadata.get('END_DATE', None).split('T')[0]
start_date_gnss = dt.strptime(start_date, "%Y%m%d")
end_date_gnss = dt.strptime(end_date, "%Y%m%d")

# Get center lat/lon of analysis region
bbox = sitedata['sites'][site]['analysis_region'].replace("'","").split()
lat0 = (float(bbox[0]) + float(bbox[1]))/2
lon0 = (float(bbox[2]) + float(bbox[3]))/2
# Identify geometry file x/y location associated with center lat/lon
geom_obj = mintpy_dir + '/inputs/geometryGeo.h5'
atr = readfile.read_attribute(geom_obj)
coord = ut.coordinate(atr, lookup_file=geom_obj)
y, x = coord.geo2radar(lat0, lon0)[0:2]
y = max(0, y);  y = min(int(atr['LENGTH'])-1, y)
x = max(0, x);  x = min(int(atr['WIDTH'])-1, x)
# Write out inclination/azimuth at specified x/y location
kwargs = dict(box=(x,y,x+1,y+1))
inc_angle = readfile.read(geom_obj, datasetName='incidenceAngle', **kwargs)[0][0,0]
az_angle  = readfile.read(geom_obj, datasetName='azimuthAngle',   **kwargs)[0][0,0]

# Set GNSS Parameters
gnss_completeness_threshold = 0.9    # percent of data timespan with valid GNSS epochs
gnss_residual_stdev_threshold = 10.  # max threshold standard deviation of residuals to linear GNSS fit (mm/yr)

# Search for collocated GNSS stations
site_names, _, _ = gnss.search_gnss(SNWE=(S,N,W,E),
                                    start_date=start_date,
                                    end_date=end_date,
                                    source=gnss_source)
site_names = [str(stn) for stn in site_names]

### 4.4. Get GNSS Position Time Series <a id='gps_ts'></a>

We download the GNSS station data, and evaluate the station data quality based on data completeness and scatter of residuals.

In [ ]:
# Empty dicts and lists to store GNSS data
gnss_stns = {}
bad_stns = {}

for site_name in site_names:
    gnss_stn = gnss.get_gnss_class(gnss_source)(site = site_name)
    gnss_stn.open(print_msg=False)

    # check if station is in masked out area
    site_lat, site_lon = gnss_stn.get_site_lat_lon()
    site_y, site_x = ut.coordinate(insar_metadata).geo2radar(site_lat, site_lon)[:2]
    masked_loc = np.isnan(insar_velocities[site_y, site_x])

    # count number of dates in time range
    dates = gnss_stn.dates
    range_days = (end_date_gnss - start_date_gnss).days
    gnss_count = np.histogram(dates, bins=[start_date_gnss, end_date_gnss])
    gnss_count = int(gnss_count[0][0])

    # get standard deviation of residuals to linear fit
    disp_los = ut.enu2los(gnss_stn.dis_e, gnss_stn.dis_n, gnss_stn.dis_u, inc_angle, az_angle)
    disp_detrended = signal.detrend(disp_los)
    stn_stdv = np.std(disp_detrended)

    # select GNSS stations based on data completeness, scatter of residuals
    # and whether they are in masked areas
    if (range_days*gnss_completeness_threshold <= gnss_count) \
        and (stn_stdv <= gnss_residual_stdev_threshold) \
        and (masked_loc == False):
        gnss_stns[site_name] = gnss_stn
    else:
        bad_stns[site_name] = gnss_stn

# [optional] manually remove additional stations
gnss_to_remove=[]

for site_name in gnss_to_remove:
    bad_stns[site_name] = gnss_stns[site_name]
    del gnss_stns[site_name]

# Final list of site names
site_names = [*gnss_stns]
bad_site_names = [*bad_stns]

print("\nFinal list of {} stations used in analysis:".format(len(site_names)))
print(site_names)
print("List of {} stations removed from analysis".format(len(bad_site_names)))
print(bad_site_names)

In [ ]:
# Plot stations in map view
fig, ax = plt.subplots(figsize=[18, 5.5])
cax = ax.imshow(insar_velocities, cmap=cmap, vmin=vmin, vmax=vmax, interpolation='nearest', extent=(W, E, S, N))
cbar = fig.colorbar(cax, ax=ax)
cbar.set_label('LOS velocity [mm/year]')

# Plot valid stations
for i, gnss_stn in enumerate(gnss_stns.values()):
    # Plot station
    ax.scatter(gnss_stn.site_lon, gnss_stn.site_lat, s=8**2, c='k',
               label=('valid' if i == 0 else None))
    ax.annotate(gnss_stn.site, (gnss_stn.site_lon, gnss_stn.site_lat))

# Plot rejected stations
for i, gnss_stn in enumerate(bad_stns.values()):
    # Plot station
    ax.scatter(gnss_stn.site_lon, gnss_stn.site_lat, s=8**2,
               marker='X', facecolor='none', edgecolor='k',
              label=('rejected' if i == 0 else None))
    ax.annotate(gnss_stn.site, (gnss_stn.site_lon, gnss_stn.site_lat))

ax.legend()


### 4.5. Estimate GNSS LOS Velocities <a id='gps_vel'></a>

We estimate the GNSS velocities in InSAR line of sight, by projecting the 3D displacements into LOS and fitting the resulting time-series using the same basis functions (i.e., linear and seasonal terms) that were used to fit the InSAR time-series. Outliers are removed from the GNSS position data and the functional fits are recomputed.

In [ ]:
# Outlier ID and removal parameters
outlier_thresh = 3  # standard deviations
outlier_iters = 1  # iterations

# Loop through valid GNSS stations
gnss_fits = {}  # empty dict to store fits
gnss_sites = {}  # empty dict to store site measurements

for i, site_name in enumerate(site_names):
    # Retrieve station information
    gnss_stn = gnss_stns[site_name]
    gnss_stn.get_los_displacement(insar_metadata,
                                  start_date=start_date,
                                  end_date=end_date)

    # Scale displacement values from m to mm
    scale_gnss_m_to_mm(gnss_stn)
    
    # Outlier detection and removal
    gnss_fit = IterativeOutlierFit(gnss_stn.dates, gnss_stn.dis_los,
                                   model=ts_functions, threshold=outlier_thresh,
                                   max_iter=outlier_iters)
    gnss_fits[site_name] = gnss_fit  # record for posterity

    # Record GNSS site velocity
    gnss_site = SiteVelocity(site=site_name,
                             site_lon=gnss_stn.site_lon,
                             site_lat=gnss_stn.site_lat,
                             vel=gnss_fit.m_hat[1],
                             vel_err=gnss_fit.mhat_se[1],
                             unit='mm/yr')
    gnss_sites[site_name] = gnss_site

    # Report
    if i == 0 :
        print('site velocity(mm/yr)')
    print(gnss_site)

    # Plotting parameters
    n_dates = len(gnss_stn.dates)
    label_skips = n_dates//6
    ax_nb = i % 2

    # Spawn figure for even numbers
    if ax_nb == 0:
        fig, axes = plt.subplots(figsize=(10, 4), ncols=2)

    # Plot outliers
    if gnss_fit.n_outliers > 0:
        axes[ax_nb].scatter(gnss_fit.outlier_dates,
                            gnss_fit.outlier_dis,
                            2**2, 'firebrick', label='outliers')

    # Plot filtered data and model fit
    axes[ax_nb].scatter(gnss_fit.dates, gnss_fit.dis, 3**2,
                        'dimgrey', zorder=1)
    axes[ax_nb].plot(gnss_fit.dates, gnss_fit.dis_hat,
                     'c', linewidth=3, label='model fit', zorder=2)
    axes[ax_nb].plot(gnss_fit.dates, gnss_fit.err_envelope[0],
                     'b--', zorder=3)
    axes[ax_nb].plot(gnss_fit.dates, gnss_fit.err_envelope[1],
                     'b--', zorder=3, label='95% conf')
    
    # Format plot
    axes[ax_nb].legend()
    axes[ax_nb].set_xticks(gnss_stn.dates[::label_skips])
    axes[ax_nb].set_xticklabels([date.strftime('%Y-%m-%d') \
                                 for date in gnss_stn.dates[::label_skips]],
                                rotation=80)
    axes[ax_nb].set_ylabel('LOS displacement (mm)')
    axes[ax_nb].set_title(gnss_site)
    fig.tight_layout()


### 4.6. Re-reference GNSS and InSAR <a id='reference'></a>

In [ ]:
# Reference GNSS stations to reference site
ref_site = sitedata['sites'][site]['gps_ref_site_name']
if sitedata['sites'][site]['gps_ref_site_name'] == 'auto':
    ref_site = [*gnss_sites.keys()][0]

gnss_ref_vel = gnss_sites[ref_site].vel
print(f"Using GNSS reference station: {ref_site}")

for site_name in site_names:
    gnss_sites[site_name].vel -= gnss_ref_vel
    print(str(gnss_sites[site_name]))

# Reference InSAR to GNSS reference site
(ref_y,
 ref_x,
 _, _) = ut.coordinate(insar_metadata).geo2radar(gnss_sites[ref_site].site_lat,
                                                 gnss_sites[ref_site].site_lon)
insar_ref_velocity = insar_velocities[ref_y, ref_x]
insar_velocities = insar_velocities - insar_ref_velocity
print(f"Insar reference velocity {insar_ref_velocity:.1f} mm/yr")

# Plot GNSS stations on InSAR velocity field
fig, ax = plt.subplots(figsize=[18, 5.5])
cax = ax.imshow(insar_velocities, cmap=cmap, vmin=vmin, vmax=vmax,
                interpolation='nearest', extent=(W, E, S, N))
cbar = fig.colorbar(cax, ax=ax)
cbar.set_label('LOS velocity [mm/year]')

for site_name in site_names:
    gnss_site = gnss_sites[site_name]
    color = cmap((gnss_site.vel - vmin)/(vmax-vmin))
    ax.scatter(gnss_site.site_lon, gnss_site.site_lat, color=color, s=8**2, edgecolors='k')
    ax.text(gnss_site.site_lon, gnss_site.site_lat, gnss_site.site)
    ax.set_title(f"{gnss_source} GNSS velocities on InSAR velocities")


<div class="alert alert-info">
<b>Note :</b> 
Negative values indicates that target is moving away from the radar (i.e., subsidence in case of vertical deformation).
Positive values indicates that target is moving towards the radar (i.e., uplift in case of vertical deformation). 
</div>

<br>
<hr>

<a id='validation1'></a>
## 5. Validation Method 1: GNSS-InSAR Direct Comparison 

### 5.1. Make GNSS-InSAR Velocity Residuals at GNSS Station Locations <a id='make_resid'></a>

In [ ]:
# Set Parameters
pixel_radius = 5   #number of InSAR pixels to average for comparison with GNSS

# Loop over InSAR measurements at GNSS station locations
insar_sites = {}  # empty dict to store site measurements

for i, site_name in enumerate(site_names):
    gnss_site = gnss_sites[site_name]
    
    # Convert GNSS station lat/lon information to InSAR x/y grid
    stn_y, stn_x, _, _ = coord.geo2radar(gnss_site.site_lat,
                                         gnss_site.site_lon)

    # Caution: If you expand the radius parameter farther than the bounding grid it will break. 
    # To fix, remove the station in section 4 when the site_names list is filtered
    vel_px_rad = insar_velocities[stn_y-pixel_radius:stn_y+1+pixel_radius, 
                                  stn_x-pixel_radius:stn_x+1+pixel_radius]
    insar_site_vel = np.nanmedian(vel_px_rad)

    # Assign to object
    insar_site = SiteVelocity(site=site_name,
                              site_lon=gnss_site.site_lon,
                              site_lat=gnss_site.site_lat,
                              vel=insar_site_vel,
                              unit='mm/yr')
    insar_sites[site_name] = insar_site

    # Report
    if i == 0 :
        print('site velocity(mm/yr)')
    print(insar_site)


### 5.2. Make Double-Differenced Velocity Residuals <a id='make_ddiff'></a>

Adjusting the InSAR velocity field to a single reference site, as done in Section 4.9, leaves room for discrepancies between the InSAR and GNSS data sets. Instead, this validation exercise focuses on the ability to accurately measure relative displacement rates across a scene. To eliminate bias introduced by imprecise registration between the InSAR and GNSS data, we difference the InSAR and GNSS relative velocities between pairs of stations, i.e., the "double-difference" between site pairs.

We compute the double-differences below.

In [ ]:
# Compute double differences between site pairs
n_sites = len(site_names)

# Loop over stations
double_diffs = {}  # empty dict to populate with InSAR - GNSS diffs
for i in range(n_sites - 1):
    stn1 = site_names[i]
    for j in range(i + 1, n_sites):
        stn2 = site_names[j]

        # Compute distance (km) between site locations
        site_dist = haversine_distance(gnss_sites[stn1].site_lon,
                                       gnss_sites[stn1].site_lat,
                                       gnss_sites[stn2].site_lon,
                                       gnss_sites[stn2].site_lat)
        
        # Calculate site-to-site differences
        diff_name = f"{stn1}-{stn2}"
        gnss_diff = gnss_sites[stn1] - gnss_sites[stn2]
        insar_diff = insar_sites[stn1] - insar_sites[stn2]
        diff_res = gnss_diff - insar_diff
        diff_res.dist = site_dist
        diff_res.site = diff_name

        # Record to dictionary
        double_diffs[diff_name] = diff_res


### 5.3. Secular Requirement Validation: Method 1 <a id='secular_validation_method1'></a>

We assume that the distribution of residuals is Gaussian and that the requirement success threshold represents a 1-sigma limit within which we expect 68.3% of residuals to lie.

In [ ]:
# Define requirement
secular_threshold_rqmt = 2  # mm/yr
secular_distance_rqmt = [0.1, 50.0]  # km

# Validation parameters
n_bins = 10
threshold = 0.683  

# Write data for statistical tests
site_dist = np.array([diff_res.dist for diff_res in double_diffs.values()])
double_diff_rel_measure = np.abs(np.array([diff_res.vel for diff_res in double_diffs.values()]))

# Validation figure and assessment
validation_table, fig = display_validation(site_dist,                           # binned distance for point
                                           double_diff_rel_measure,             # binned double-difference velocities mm/yr
                                           site,                                # cal/val site name
                                           start_date,                          # start date of InSAR dataset
                                           end_date,                            # end date of InSAR dataset 
                                           requirement=secular_threshold_rqmt,  # measurement requirement to meet, e.g 2 mm/yr for 3 years of data over 0.1-50km
                                           distance_rqmt=secular_distance_rqmt, # distance over requirement is to meet, e.g. over length scales of 0.1-50 km [0.1, 50] 
                                           n_bins=n_bins,                       # number of bins, to collect statistics 
                                           threshold=threshold,                 # quantile threshold for point-pairs that pass requirement, e.g. 0.683 - we expect 68.3% of residuals to lie. 
                                           sensor='Sentinel-1',                 # sensor that is validated, Sentinel-1 or NISAR
                                           validation_type='secular',           # validation for: secular, transient, coseismic requirement
                                           validation_data='GNSS')              # validation method: GNSS - Method 1, InSAR - Method 2

out_fig = os.path.abspath('secular_insar-gnss_velocity_vs_distance.png')
fig.savefig(out_fig, bbox_inches='tight', transparent=True, dpi=30)

In [ ]:
display_validation_table(validation_table)

<div class="alert alert-warning">
Validation Method 1: success for a baseline distance bin occurs when the percentage of residuals within the requirement success threshold is >0.683
</div>

<br>
<hr>

<a id='validation2'></a>
## 6. Validation Method 2: InSAR-only Structure Function

In Validation Method 2, we use a time interval and area where we assume no deformation.  As with Method 1, we assume that the distribution of residuals is Gaussian and that the requirement success threshold represents a 1-sigma limit within which we expect 68.3% of residuals to lie.

### 6.1. Read InSAR Array and Mask Pixels with no Data <a id='array_mask'></a>

In [ ]:
# Use the assumed non-earthquake displacement as the insar_displacment for statistics and convert to mm
insar_velocities,_ = readfile.read(vel_file, datasetName = 'velocity')  #read velocity
velStart = sitedata['sites'][site]['download_start_date']
insar_velocities = insar_velocities * 1000.  # convert velocities from m to mm

# Set masked pixels to NaN
msk,_ = readfile.read(msk_file)
insar_velocities[msk == 0] = np.nan
insar_velocities[insar_velocities == 0] = np.nan

# Display map of velocity data after masking
cmap = plt.get_cmap('RdBu_r')
fig, ax = plt.subplots(figsize=[18, 5.5])
img1 = ax.imshow(insar_velocities, cmap=cmap, vmin=vmin, vmax=vmax, interpolation='nearest', extent=(W, E, S, N))
ax.set_title("Secular \n Date "+velStart)
cbar1 = fig.colorbar(img1, ax=ax)
cbar1.set_label('LOS velocity [mm/year]')

### 6.2. Randomly Sample Pixels and Pair Them Up with Option to Remove Trend <a id='remove_trend'></a>

In [ ]:
# Define requirement
secular_threshold_rqmt = 2  # mm/yr
secular_distance_rqmt = [0.1, 50.0]  # km

sample_mode = 'points'  # 'points' or 'profile'
# note that the 'profile' method may take significantly longer

# Collect samples using the specified method
if sample_mode in ['points']:
    X0,Y0 = load_geo(insar_metadata)
    X0_2d,Y0_2d = np.meshgrid(X0,Y0)

    insar_sample_dist, insar_rel_measure = samp_pair(X0_2d, Y0_2d, insar_velocities, num_samples=1000000)

elif sample_mode in ['profile']:
    # Sample grid setup
    length, width = int(insar_metadata['LENGTH']), int(insar_metadata['WIDTH'])
    X = np.linspace(W+lon_step, E-lon_step, width)  # longitudes
    Y = np.linspace(N+lat_step, S-lat_step, length)  # latitudes
    X_coords, Y_coords = np.meshgrid(X, Y)

    # Draw random samples from map (without replacement)
    num_samples = 20000
    
    # Retrieve profile samples
    insar_sample_dist, insar_rel_measure = profile_samples(\
                    x=X_coords.reshape(-1,1),
                    y=Y_coords.reshape(-1,1),
                    data=insar_velocities,
                    metadata=insar_metadata,
                    len_rqmt=secular_distance_rqmt,
                    num_samples=num_samples)

### 6.3. Secular Requirement Validation: Method 2  <a id='secular_validation_method2'></a>

In [ ]:
# Validation parameters
n_bins = 10
threshold = 0.683  

# Validation figure and assessment
validation_table, fig = display_validation(insar_sample_dist,              # binned distance for point
                                           insar_rel_measure,              # binned relative velocities mm/yr
                                           site,                           # cal/val site name
                                           start_date,                     # start date of InSAR dataset
                                           end_date,                       # end date of InSAR dataset 
                                           requirement=secular_threshold_rqmt,  # measurement requirement to meet, e.g 2 mm/yr for 3 years of data over 0.1-50km
                                           distance_rqmt=secular_distance_rqmt,  # distance over requirement is to meet, e.g. over length scales of 0.1-50 km [0.1, 50] 
                                           n_bins=n_bins,                  # number of bins, to collect statistics 
                                           threshold=threshold,            # quantile threshold for point-pairs that pass requirement, e.g. 0.683 - we expect 68.3% of residuals to lie. 
                                           sensor='Sentinel-1',            # sensor that is validated, Sentinel-1 or NISAR
                                           validation_type='secular',      # validation for: secular, transient, coseismic requirement
                                           validation_data='InSAR')        # validation method: GNSS - Method 1, InSAR - Method 2

out_fig = os.path.abspath('secular_insar-only_vs_distance_'+site+'_date'+velStart+'.png')
fig.savefig(out_fig, bbox_inches='tight', transparent=True, dpi=300)

In [ ]:
display_validation_table(validation_table)

<div class="alert alert-warning">
Final result Method 2—
    68% of points below the requirements line is success
</div>


<br>
<hr>

<a id='appendix'></a>
## Appendix: Supplementary Comparisons and Plots

### A.1. Compare Raw Velocities <a id='compare_raw'></a>

In [ ]:
plt.figure(figsize=(10,3))
plt.hist([insar_site.vel for insar_site in insar_sites.values()],
         range=[vmin, vmax], bins=50, color="green", edgecolor='grey', label='V_InSAR')
plt.hist([gnss_site.vel for gnss_site in gnss_sites.values()],
         range=[vmin, vmax], bins=50, color="orange", edgecolor='grey', label='V_gnss', alpha=0.5)
plt.legend(loc='upper right')
plt.title(f'''Velocities \n Date range {start_date}-{end_date}
Reference stn: {sitedata['sites'][site]['gps_ref_site_name']}
Number of stations used: {len(site_names)}''')
plt.xlabel('LOS Velocity (mm/year)')
plt.ylabel('N Stations')
plt.ylim(0, 20)
plt.show()

### A.2. Plot Velocity Residuals <a id='plot_residuals'></a>

In [ ]:
plt.figure(figsize=(10,3))
plt.hist([(insar_sites[site_name] - gnss_sites[site_name]).vel for site_name in site_names],
         bins = 40, range=[vmin,vmax], edgecolor='grey', color="darkblue", linewidth=1,
         label='V_gnss - V_InSAR (area average)')
plt.legend(loc='upper right')
plt.title(f'''Residuals \n Date range {start_date}-{end_date}
Reference stn: {sitedata['sites'][site]['gps_ref_site_name']}
Number of stations used: {len(site_names)}''')
plt.xlabel('Velocity Residual (mm/year)')
plt.ylabel('N Stations')
plt.show()

### A.3. Plot Double Difference Residuals <a id='plot_ddiff'></a>

In [ ]:
plt.figure(figsize=(10,3))
plt.hist([diff_res.vel for diff_res in double_diffs.values()],
         range = [vmin, vmax],bins = 40, color = "darkblue",edgecolor='grey',
         label='V_gnss_(s1-s2) - V_InSAR_(s1-s2)')
plt.legend(loc='upper right')
plt.title(f'''Difference Residualts \n Date range {start_date}-{end_date}
Reference stn: {sitedata['sites'][site]['gps_ref_site_name']}
Number of stations used: {len(site_names)}''')
plt.xlabel('Double Differenced Velocity Residual (mm/year)')
plt.ylabel('N Stations')
plt.show()

### A.4. GNSS Timeseries Plots <a id='plot_gps_tseries'></a>

In [ ]:
# Grab the time-series file used for time function estimation given the template setup
template = readfile.read_template(os.path.join(mintpy_dir, 'smallbaselineApp.cfg'))
template = ut.check_template_auto_value(template)
ts_file = TimeSeriesAnalysis.get_timeseries_filename(template, mintpy_dir)['velocity']['input']

# Read the time-series file
insar_ts, ts_metadata = readfile.read(ts_file, datasetName='timeseries')
mask = readfile.read(os.path.join(mintpy_dir, 'maskTempCoh.h5'))[0]
print(f"reading timeseries from file: {ts_file}")

# Get date list
date_list = timeseries(ts_file).get_date_list()
num_date = len(date_list)
date0, date1 = date_list[0], date_list[-1]
insar_dates = ptime.date_list2vector(date_list)[0]

# Spatial reference
coord = ut.coordinate(ts_metadata)
ref_site = sitedata['sites'][site]['gps_ref_site_name']
ref_gnss_obj = gnss_stns[ref_site]
ref_lat, ref_lon = ref_gnss_obj.get_site_lat_lon()
ref_y, ref_x = coord.geo2radar(ref_lat, ref_lon)[:2]
if not mask[ref_y, ref_x]:
    raise ValueError(f'Given reference GNSS site ({ref_site}) is in mask-out unrelible region in InSAR! Change to a different site.')
ref_insar_dis = insar_ts[:, ref_y, ref_x]

# Plot displacements and velocity timeseries at GNSS station locations
num_site = len(site_names)
prog_bar = ptime.progressBar(maxValue=num_site)
for i, site_name in enumerate(site_names):
    prog_bar.update(i+1, suffix=f'{site_name} {i+1}/{num_site}')

    ## Read data
    # Recall gnss station displacements with outliers removed
    gnss_obj = gnss_stns[site_name]
    gnss_lalo = (gnss_obj.site_lat, gnss_obj.site_lon)

    # Get relative LOS displacement on common dates
    gnss_dates = np.array(sorted(list(set(gnss_obj.dates) & set(ref_gnss_obj.dates))))
    gnss_dis = np.zeros(gnss_dates.shape, dtype=np.float32)
    for i, date_i in enumerate(gnss_dates):
        idx1 = np.where(gnss_obj.dates == date_i)[0][0]
        idx2 = np.where(ref_gnss_obj.dates == date_i)[0][0]
        gnss_dis[i] = gnss_obj.dis_los[idx1] - ref_gnss_obj.dis_los[idx2]
    
    # Shift GNSS to zero-mean in time [for plotting purpose]
    gnss_dis -= np.nanmedian(gnss_dis)

    # Read InSAR
    y, x = coord.geo2radar(gnss_lalo[0], gnss_lalo[1])[:2]
    insar_dis = insar_ts[:, y, x] - ref_insar_dis

    # Apply a constant shift in time to fit InSAR to GNSS
    comm_dates = sorted(list(set(gnss_dates) & set(insar_dates)))
    if comm_dates:
        insar_flag = [x in comm_dates for x in insar_dates]
        gnss_flag = [x in comm_dates for x in gnss_dates]
        insar_dis -= np.nanmedian(insar_dis[insar_flag] - gnss_dis[gnss_flag])

    ## Plot figure
    if gnss_dis.size > 0 and np.any(~np.isnan(insar_dis)):
        fig, ax = plt.subplots(figsize=(10, 3))
        ax.axhline(color='grey',linestyle='dashed', linewidth=2)
        ax.scatter(gnss_dates, gnss_dis*1000, s=2**2, label="GNSS Daily Positions")
        ax.scatter(insar_dates, insar_dis*1000, label="InSAR Positions")
        # axis format
        ax.set_title(f"Station Name: {site_name}") 
        ax.set_ylabel('LOS displacement [mm]')
        ax.legend()
prog_bar.close()
plt.show()